# Classifier Selection 

Comparing multiple dimensionality reduction techniques on the full dataset, it seems that it might be possible to find a linear classifier that seperates these classes.  I'm going to give two basic classifiers a shot on both the full data and the DMN subset. However, since I have only $200$ samples I'm going to need to do some unsupervised dimensionality reduction on the full set of autocorrelation values. 

In [1]:

import sys
sys.path.append('..') #workaround to deal with directory issues in notebooks

import numpy as np
import pandas as pd 

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import (KNeighborsClassifier,
                               NeighborhoodComponentsAnalysis)

from xgboost import XGBClassifier

from src.models import train_model 
from src.features import load_features


In [2]:
data_dir='../data/'
class_labels,two_class_labels,pos_str,neg_str,clus_co,ar_array,num_regions,num_subjs=load_features(data_dir)

In [3]:
x=np.concatenate((ar_array,pos_str,neg_str,clus_co),axis=0).transpose() # sets up correctly for scikit learn input later


#do model selection based on a test train split
x_train, x_test, y_train, y_test = train_test_split(x, two_class_labels, test_size=.20, random_state=42,stratify=two_class_labels)

cv=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scale=StandardScaler()
x_train_sc=scale.fit_transform(x_train)
x_test_sc=scale.transform(x_test)

#pipelines=[pipe_svc,pipe_XGB,pipe_pca_svc,pipe_pca_XGB,pipe_MI_svc,pipe_MI_XGB]

pipelines=["svc_0.1","xgb_0.1","pca_svc_0.1","pca_xgb_0.1"] #current (crude) model versioning

subsets={"Auto":np.array([*range(num_regions)]),"Pos":np.array([*range(num_regions,2*num_regions)]),"Neg":np.array([*range(2*num_regions,3*num_regions)]),"Clus":np.array([*range(3*num_regions,4*num_regions)]),"All":np.array([*range(4*num_regions)]),"AutoPos":np.array([*range(2*num_regions)])}



In [4]:
fit_models,score,estimator=train_model.train_multi_subset_pipeline(x_train_sc,y_train,cv,subsets,pipelines,save_flag=True)

In [5]:
performance=train_model.score_on_test(x_test_sc,y_test,fit_models,subsets)

In [6]:
print(performance)

               Pipeline     Score  Variance
0          Auto_svc_0.1  0.539886  0.003532
1          Auto_xgb_0.1  0.445869  0.003032
2      Auto_pca_svc_0.1  0.465812  0.004547
3      Auto_pca_xgb_0.1  0.430199  0.006177
4           Pos_svc_0.1  0.616809  0.005355
5           Pos_xgb_0.1  0.447293  0.004734
6       Pos_pca_svc_0.1  0.519943  0.001910
7       Pos_pca_xgb_0.1  0.391738  0.005135
8           Neg_svc_0.1  0.467236  0.005936
9           Neg_xgb_0.1  0.502849  0.004093
10      Neg_pca_svc_0.1  0.500000  0.000000
11      Neg_pca_xgb_0.1  0.465812  0.004547
12         Clus_svc_0.1  0.698006  0.006717
13         Clus_xgb_0.1  0.444444  0.000962
14     Clus_pca_svc_0.1  0.500000  0.000000
15     Clus_pca_xgb_0.1  0.428775  0.004894
16          All_svc_0.1  0.559829  0.004868
17          All_xgb_0.1  0.502849  0.004093
18      All_pca_svc_0.1  0.524217  0.006557
19      All_pca_xgb_0.1  0.501425  0.002230
20      AutoPos_svc_0.1  0.501425  0.002230
21      AutoPos_xgb_0.1  0.50142

In [7]:
performance.to_csv('Model_Performance_0.1.csv')
